<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Aplicada/blob/main/Script_Clase_28_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎨 Inteligencia Artificial Aplicada
## Universidad de los Andes

### 👨‍🏫 Profesores
- **Profesor Magistral:** [Camilo Vega Barbosa](https://www.linkedin.com/in/camilovegabarbosa/)
- **Asistente de Docencia:** [Sergio Julian Zona Moreno](https://www.linkedin.com/in/sergiozonamoreno/)

### 📚 Fine-Tuning con PEFT-LoRA para Modelos de Lenguaje
Este script implementa un proceso de fine-tuning eficiente para modelos de lenguaje:

1. **Configuración del Modelo y Datos 🚀**
   - Uso de Microsoft Phi-2, un modelo de 2.7B de parámetros con alta capacidad
   - Carga de datos especializados para aprender el "idioma 4" (reemplazar "a" por "4")
   - Preparación eficiente de tokens y formato de instrucción
   - Integración con GitHub para obtener el dataset directamente desde repositorio

2. **Implementación de PEFT-LoRA 🧠**
   - Parameter-Efficient Fine-Tuning (PEFT) con Low-Rank Adaptation (LoRA)
   - Modificación selectiva de solo ~1% de los parámetros del modelo
   - Enfoque en capas estratégicas: q_proj, k_proj, v_proj, o_proj, fc1, fc2
   - Cuantización de 8-bits para reducción drástica de memoria requerida
   - Preservación de los pesos originales del modelo para mantener capacidades base

3. **Proceso de Entrenamiento Optimizado 📈**
   - Configuración adaptable: modo rápido para pruebas y modo profundo para producción
   - Visualización en tiempo real de la curva de pérdida para monitoreo del aprendizaje
   - Sistema de callbacks personalizados para seguimiento del progreso
   - Guardado eficiente del modelo adaptado con solo los parámetros LoRA (~MB vs ~GB)

4. **Inferencia y Aplicación Interactiva 🔍**
   - Interfaz Gradio para pruebas del modelo con cualquier texto de entrada
   - Generación controlada con parámetros ajustables (temperatura, top_p)
   - Ejemplos predefinidos para demostración inmediata
   - Despliegue web temporal para compartir el modelo con otros usuarios



In [ ]:
# Instalamos las bibliotecas necesarias
!pip install -q transformers datasets peft bitsandbytes accelerate gradio

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# 1. CARGA DE DATOS
# Descargamos los datos directamente desde el repositorio de GitHub
# Definimos la ruta al archivo CSV en tu repositorio
github_repo = "CamiloVga/Curso-IA-Aplicada"
branch = "main"
file_path = "Semana 14_Fine-Tuning y RAG/Base Fine-Tuning Idioma 4.csv"

# Cargamos el dataset desde GitHub
print("Cargando dataset desde GitHub...")
dataset = load_dataset("csv",
                      data_files=f"https://raw.githubusercontent.com/{github_repo}/{branch}/{file_path}")

# Exploramos el dataset
print("Estructura del dataset:")
print(dataset)
print("\nEjemplo de entrada:")
print(dataset["train"][0])
print(f"Número de ejemplos: {len(dataset['train'])}")

# 2. PREPROCESAMIENTO DE DATOS
# Phi-2 es un modelo más pequeño (2.7B) pero de alto rendimiento y acceso abierto
model_name = "microsoft/phi-2"

print(f"Cargando tokenizador para {model_name}...")
# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Aseguramos que el tokenizador tenga tokens especiales necesarios
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Definimos una función para preprocesar los datos
def preprocess_function(examples):
    # Formateamos los textos como instrucciones siguiendo un formato tipo chat
    prompts = []
    for i in range(len(examples["input"])):
        # Formato de instrucción para Phi-2
        prompt = f"<s>Traduce este texto al idioma 4: {examples['input'][i]}\n\n{examples['output'][i]}</s>"
        prompts.append(prompt)

    # Tokenizamos los textos
    tokenized_inputs = tokenizer(
        prompts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    # Configuramos las labels igual que los input_ids para entrenamiento de LM causal
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].clone()

    return tokenized_inputs

# Aplicamos la función de preprocesamiento al dataset
print("Procesando y tokenizando el dataset...")
tokenized_dataset = dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(f"Dataset tokenizado: {tokenized_dataset}")

# 3. CONFIGURACIÓN DEL MODELO
# Configuramos BitsAndBytes para cuantización de 8 bits (ahorra memoria)
print("Configurando cuantización para el modelo...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Phi-2 funciona bien con cuantización de 8 bits
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    bnb_8bit_compute_dtype=torch.float16
)

# Cargamos el modelo base con cuantización
print(f"Cargando modelo {model_name} con cuantización...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Preparamos el modelo para entrenamiento con LoRA
model = prepare_model_for_kbit_training(model)

# 4. CONFIGURACIÓN DE LORA
# Para Phi-2, ajustamos los target_modules a su arquitectura específica
print("Configurando adaptadores LoRA...")
peft_config = LoraConfig(
    r=16,  # Dimensión del adaptador LoRA - Determina el tamaño de las matrices de bajo rango
           # Valores más altos aumentan la capacidad pero también el número de parámetros
           # Típicamente entre 8-64 dependiendo de la complejidad de la tarea

    lora_alpha=32,  # Parámetro de escala - Factor que controla la magnitud de la actualización LoRA
                    # Generalmente se establece como 2*r para un buen equilibrio
                    # Valores más altos = mayor impacto de las actualizaciones LoRA

    lora_dropout=0.05,  # Regularización - Aplica dropout a las capas LoRA para prevenir sobreajuste
                        # Valores típicos entre 0.01-0.1

    bias="none",  # Configuración para bias - "none" significa que no se entrenan los sesgos
                  # Otras opciones: "all" (todos los sesgos) o "lora_only" (solo sesgos en capas LoRA)

    task_type="CAUSAL_LM",  # Tipo de tarea - Modelado de lenguaje causal (generación de texto)
                           # Otras opciones: "SEQ_CLS" (clasificación), "SEQ_2_SEQ_LM" (seq2seq), etc.

    # Ajustamos los módulos objetivo según la arquitectura de Phi-2
    # Estas son las únicas capas del modelo que se modificarán durante el entrenamiento
    target_modules=[
        "q_proj",  # Proyección de queries en los bloques de atención
                  # Transforma los vectores de entrada en queries para la auto-atención

        "k_proj",  # Proyección de keys en los bloques de atención
                  # Transforma los vectores de entrada en keys para la auto-atención

        "v_proj",  # Proyección de values en los bloques de atención
                  # Transforma los vectores de entrada en values para la auto-atención

        "o_proj",  # Proyección de output en los bloques de atención
                  # Combina los resultados de la atención para la salida

        "fc1",    # Primera capa feed-forward en los bloques del transformador
                 # Expande la dimensionalidad (MLP)

        "fc2"     # Segunda capa feed-forward en los bloques del transformador
                 # Reduce la dimensionalidad de vuelta (MLP)
    ]
)

# Convertimos el modelo a un modelo PEFT usando LoRA
# Esto añade adaptadores de bajo rango a las capas especificadas
# sin modificar los pesos originales del modelo
model = get_peft_model(model, peft_config)

# Imprimir parámetros entrenables vs totales
# Esto mostrará la eficiencia de LoRA - típicamente solo entrena <1% de parámetros
print("Resumen de parámetros del modelo:")
model.print_trainable_parameters()




In [ ]:
import pandas as pd
info=pd.DataFrame(dataset)
info

In [ ]:
# 5. ENTRENAMIENTO
# Configuramos los argumentos de entrenamiento para un entrenamiento rápido inicial
print("Configurando parámetros de entrenamiento...")
training_args = TrainingArguments(
    output_dir="./results_idioma4_phi2",
    # Configuración para entrenamiento rápido
    per_device_train_batch_size=8,         # Tamaño de batch más grande para velocidad
                                          # Para entrenamiento profundo: reducir a 2-4

    gradient_accumulation_steps=2,        # Menos pasos de acumulación para velocidad
                                          # Para entrenamiento profundo: aumentar a 8-16

    warmup_steps=5,                       # Menos pasos de calentamiento
                                          # Para entrenamiento profundo: aumentar a 50-100

    max_steps=50,                         # Menos pasos totales para prueba rápida
                                          # Para entrenamiento profundo: aumentar a 500-1000

    learning_rate=3e-4,                   # Learning rate más alto para convergencia rápida
                                          # Para entrenamiento profundo: reducir a 1e-4 o 5e-5

    fp16=True,                            # Usar precisión mixta para acelerar el entrenamiento

    logging_steps=5,                      # Registro frecuente para ver progreso

    save_steps=5,                         # Guardar checkpoints
                                          # Para entrenamiento profundo: cada 100-200 pasos

    save_total_limit=2,                   # Mantener menos checkpoints para ahorrar espacio

    report_to="none",                     # No enviar métricas a servicios externos

    # Parámetros adicionales para entrenamiento más estable
    # Descomentar para entrenamiento profundo:
    # weight_decay=0.01,                  # Regularización L2 para evitar sobreajuste
    # lr_scheduler_type="cosine",         # Programación de tasa de aprendizaje tipo coseno
    # max_grad_norm=1.0,                  # Recorte de gradiente para estabilidad
)

# Creamos un callback personalizado para registrar las pérdidas durante el entrenamiento
from transformers import TrainerCallback
import matplotlib.pyplot as plt

class LossCallback(TrainerCallback):
    def __init__(self):
        self.losses = []
        self.steps = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None and "loss" in logs:
            self.losses.append(logs["loss"])
            self.steps.append(state.global_step)

    def plot_loss(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.steps, self.losses, marker='o', linestyle='-', color='blue')
        plt.title('Pérdida durante el entrenamiento', fontsize=16)
        plt.xlabel('Pasos de entrenamiento', fontsize=14)
        plt.ylabel('Pérdida', fontsize=14)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig('loss_plot.png')  # Guardar la gráfica como imagen
        plt.show()

# Instanciamos el callback
loss_callback = LossCallback()

# Collator de datos para el entrenamiento
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No es masked language modeling, sino causal
)

# Inicializamos el Trainer con nuestro callback
print("Inicializando Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    callbacks=[loss_callback]  # Añadimos nuestro callback para registrar las pérdidas
)

# Entrenamos el modelo
print("Comenzando entrenamiento...")
trainer.train()
print("Entrenamiento completado.")

# Graficamos la pérdida
print("Generando gráfica de pérdida...")
loss_callback.plot_loss()

# 6. GUARDADO DEL MODELO
# Guardamos el modelo adaptado con LoRA
print("Guardando modelo y tokenizador...")
model.save_pretrained("./idioma4_phi2_lora")
tokenizer.save_pretrained("./idioma4_phi2_lora")
print("Modelo guardado en './idioma4_phi2_lora'")



In [ ]:
# 7. INFERENCIA CON GRADIO

import gradio as gr

def generate_idioma4(input_text, max_length=200):
    """Traduce texto al idioma 4 usando el modelo fine-tuneado."""
    # Manejamos el caso de texto vacío
    if not input_text.strip():
        return "Por favor ingresa algún texto para traducir."

    # Preparamos el prompt según el formato usado en entrenamiento
    prompt = f"<s>Traduce este texto al idioma 4: {input_text}\n\n"

    # Tokenizamos
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generamos la respuesta
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            temperature=0.7,        # Controla la aleatoriedad (más bajo = más determinista)
            top_p=0.9,              # Muestreo nucleus - considera tokens con probabilidad acumulada de 0.9
            do_sample=True,         # Muestreo probabilístico en lugar de greedy decoding
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decodificamos la respuesta
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraemos solo la parte de respuesta (después del doble salto de línea)
    if "\n\n" in generated_text:
        response = generated_text.split("\n\n")[1].strip()
    else:
        response = generated_text.replace(prompt, "").strip()

    return response

# Creamos la interfaz con Gradio
def create_gradio_interface():
    """Crea y lanza una interfaz Gradio para la traducción al idioma 4."""

    # Definimos la interfaz
    demo = gr.Interface(
        fn=generate_idioma4,              # Función que procesa la entrada
        inputs=[
            gr.Textbox(
                placeholder="Escribe texto para traducir al idioma 4...",
                label="Texto Original",
                lines=5
            )
        ],
        outputs=[
            gr.Textbox(label="Traducción al Idioma 4", lines=5)
        ],
        title="Traductor al Idioma 4",
        description="""
        <h3>¡Bienvenido al Traductor de Idioma 4!</h3>
        <p>Este es un modelo de lenguaje fine-tuneado con PEFT-LoRA para traducir texto al "idioma 4",
        donde cada letra 'a' se reemplaza por el número '4'.</p>
        <p><b>Ejemplo:</b> "La casa amarilla" → "L4 c4s4 4m4rill4"</p>
        """,
        examples=[
            ["Hola a todos, me llamo Camilo"],
            ["La casa amarilla está en la playa"],
            ["Vamos a aprender inteligencia artificial"],
            ["El agua clara cae desde la alta cascada"]
        ],
        theme=gr.themes.Soft()  # Tema visual más atractivo
    )

    # Lanzamos la interfaz
    demo.launch(share=True)  # share=True crea un enlace compartible
    return demo

# Lanzamos la interfaz Gradio
print("Iniciando interfaz Gradio para el traductor de Idioma 4...")
interface = create_gradio_interface()